In [3]:
import json
import signal
import sys
from confluent_kafka import Producer


In [4]:
# config_common.py
kafka_conf = {
    "bootstrap.servers": "kafka.dongango.com:9094",
    # 필요 시 아래 주석 해제하여 SASL/SCRAM 사용
    # "security.protocol": "SASL_SSL",  # 또는 "SASL_PLAINTEXT"
    # "sasl.mechanisms": "SCRAM-SHA-512",
    # "sasl.username": "your_user",
    # "sasl.password": "your_password",
}

KAFKA_TOPIC = "zolgima-test"

# 예시 JSON 페이로드 (졸음지표 샘플)
sample_event = {
    "driver_id": "demo-001",
    "ts": "2025-08-27T09:15:00+09:00",
    "metrics": {
        "EAR": 0.19,
        "MAR": 0.45,
        "blink_rate_per_min": 24,
        "yawn_rate_per_min": 2
    },
    "source": "webcam-01",
    "frame_id": 12345,
}


In [5]:
# producer.py
running = True

def handle_sigint(signum, frame):
    global running
    running = False

def delivery_report(err, msg):
    if err is not None:
        print(f"❌ Delivery failed: {err}")
    else:
        print(f"✅ Delivered to {msg.topic()} [{msg.partition()}] @ offset {msg.offset()}")

def main():
    # 약간의 튜닝을 추가해도 좋습니다.
    conf = {
        **kafka_conf,
        "linger.ms": 10,            # 소량 배치
        "batch.num.messages": 1000,
        "compression.type": "lz4",  # 네트워크 절약
    }
    p = Producer(conf)

    signal.signal(signal.SIGINT, handle_sigint)

    try:
        # 예시로 5건 전송
        for i in range(5):
            payload = {**sample_event, "seq": i}
            value_bytes = json.dumps(payload, ensure_ascii=False).encode("utf-8")
            p.produce(
                topic=KAFKA_TOPIC,
                value=value_bytes,
                key=str(payload["driver_id"]).encode("utf-8"),
                callback=delivery_report,
            )
            # 내부 큐 드레인
            p.poll(0)

        print("⏳ Flushing...")
        p.flush(10)  # 최대 10초 대기
        print("✔ Done.")
    finally:
        # 종료 시 남은 메시지 전송 시도
        try:
            p.flush(5)
        except Exception:
            pass

if __name__ == "__main__":
    main()


⏳ Flushing...
✅ Delivered to zolgima-test [0] @ offset 0
✅ Delivered to zolgima-test [0] @ offset 1
✅ Delivered to zolgima-test [0] @ offset 2
✅ Delivered to zolgima-test [0] @ offset 3
✅ Delivered to zolgima-test [0] @ offset 4
✔ Done.


In [6]:
# consumer.py
import json
import signal
import sys
from confluent_kafka import Consumer, KafkaException, KafkaError


running = True

def handle_sigint(signum, frame):
    global running
    running = False

def main():
    conf = {
        **kafka_conf,
        "group.id": "zolgima-test-consumer-group",
        "enable.auto.commit": True,
        "auto.offset.reset": "earliest",  # 처음부터 읽고 싶을 때
        "session.timeout.ms": 45000,
    }

    c = Consumer(conf)
    c.subscribe([KAFKA_TOPIC])

    signal.signal(signal.SIGINT, handle_sigint)
    print(f"👂 Subscribed to {KAFKA_TOPIC}. Press Ctrl+C to stop.")

    try:
        while running:
            msg = c.poll(1.0)  # 1초 대기
            if msg is None:
                continue
            if msg.error():
                # 파티션 EOF 등 무해한 에러 처리
                if msg.error().code() == KafkaError._PARTITION_EOF:
                    continue
                raise KafkaException(msg.error())

            # 정상 메시지 처리
            try:
                value = msg.value()
                data = json.loads(value.decode("utf-8")) if value else None
                key = msg.key().decode("utf-8") if msg.key() else None
                print(
                    f"[{msg.topic()} p{msg.partition()} @ {msg.offset()}] "
                    f"key={key} data={data}"
                )
            except json.JSONDecodeError as e:
                print(f"⚠ JSON decode error: {e}; raw={msg.value()!r}")
    finally:
        print("🛑 Closing consumer...")
        c.close()

if __name__ == "__main__":
    main()


👂 Subscribed to zolgima-test. Press Ctrl+C to stop.
[zolgima-test p0 @ 0] key=demo-001 data={'driver_id': 'demo-001', 'ts': '2025-08-27T09:15:00+09:00', 'metrics': {'EAR': 0.19, 'MAR': 0.45, 'blink_rate_per_min': 24, 'yawn_rate_per_min': 2}, 'source': 'webcam-01', 'frame_id': 12345, 'seq': 0}
[zolgima-test p0 @ 1] key=demo-001 data={'driver_id': 'demo-001', 'ts': '2025-08-27T09:15:00+09:00', 'metrics': {'EAR': 0.19, 'MAR': 0.45, 'blink_rate_per_min': 24, 'yawn_rate_per_min': 2}, 'source': 'webcam-01', 'frame_id': 12345, 'seq': 1}
[zolgima-test p0 @ 2] key=demo-001 data={'driver_id': 'demo-001', 'ts': '2025-08-27T09:15:00+09:00', 'metrics': {'EAR': 0.19, 'MAR': 0.45, 'blink_rate_per_min': 24, 'yawn_rate_per_min': 2}, 'source': 'webcam-01', 'frame_id': 12345, 'seq': 2}
[zolgima-test p0 @ 3] key=demo-001 data={'driver_id': 'demo-001', 'ts': '2025-08-27T09:15:00+09:00', 'metrics': {'EAR': 0.19, 'MAR': 0.45, 'blink_rate_per_min': 24, 'yawn_rate_per_min': 2}, 'source': 'webcam-01', 'frame_id